In [1]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import urllib.request as urllib
from requests import get
import matplotlib.pyplot as plt
from dateutil import parser
%matplotlib inline

### Extracting the list of links of items

In [17]:
base_url = 'https://www.ebay.com/b/Camping-Hiking-Equipment/16034/bn_1959887?rt=nc&_pgn='
## this function returns a list of links correspoding to an item on sale in ebay
def get_item_links(base_url, uppercut = 800000):
    links = []
    count = 1
    while len(links)<uppercut:
        url = base_url+str(count)
        response = get(url)
        print(url)
        html_soup = BeautifulSoup(response.text, 'html.parser')
        item_page_links = html_soup.find_all('div', class_ = "s-item__wrapper clearfix")
        for link in item_page_links:
            links.append(link.a.attrs['href'])
        count+=1
    return links[:uppercut]
    

#print(response.text[500:1000])

In [18]:
all_items_link = get_item_links(base_url=base_url, uppercut=40)

https://www.ebay.com/b/Camping-Hiking-Equipment/16034/bn_1959887?rt=nc&_pgn=1


In [19]:
all_items_link

['https://www.ebay.com/itm/50000lm-Military-T6-LED-Flashlight-Torch-5-Modes-Zoom-Tactical-18650-Lamp-Light/372283200530?epid=2222957697&hash=item56adced412:g:SkkAAOSwU4FZ6ZCs',
 'https://www.ebay.com/itm/Mini-5000lm-Q5-LED-Tactical-Shadowhawk-X800-Flashlight-Aluminium-Torch-new/312118553320?epid=16011128007&hash=item48abb72ae8:g:MFgAAOSwevlaFa8N',
 'https://www.ebay.com/itm/Inflatable-Air-Sofa-Bed-Lazy-Sleeping-Camping-Bag-Beach-Hangout-Couch-Windbed-18/222969236800?epid=715030241&hash=item33ea008d40:m:mSeLLE_HXu8OCpTWY6uyAZA&var=521774423593',
 'https://www.ebay.com/itm/25-50-100FT-550-Paracord-Rope-7-strand-Parachute-CordCAMPING-HiKING/173323995376?hash=item285aea48f0:m:mlfroseGfePee76ORN6V5GQ&var=472035693953',
 'https://www.ebay.com/itm/Outdoor-Camping-Hiking-Survival-Bag-Travel-Emergency-Rescue-First-Aid-Kit-New/202181722317?epid=594562826&hash=item2f12f840cd:g:1cEAAOSwE8daValt',
 'https://www.ebay.com/itm/Camping-Emergency-Rod-Flint-Stone-Fire-Starter-Striker-Lighter-Survival-Out

In [23]:
def sale_rate_tables(table, table_no = 5):
    rows =  table[table_no].find_all('tr')
    sold_quantity = []
    date_of_sale = []
    for i in range(1, len(rows)-2):
        quantity = np.int(rows[i].find_all('td')[-3].text)
        sold_quantity.append(quantity)
        date_sale = parser.parse(rows[i].find_all('td')[-2].text)
        date_of_sale.append(date_sale)

    sale_duration = (date_of_sale[0]-date_of_sale[-1]).days
    total_quantity_sold = np.array(sold_quantity).sum()
    print(total_quantity_sold, sale_duration)
    sale_rate = total_quantity_sold/sale_duration
    return sale_rate
    


### Extracting data from all the item links generated previously

In [28]:
def get_item_details(list_links):
    item_description = []
    seller_name = []
    sellerfeedback = []
    no_of_sold_items = []
    item_rating = []
    rate_of_sale = []
    keyword = []
    quantity_available = []
    price = []
    currency = []
    for url in list_links:#goes through all the links
        print(url)
        response = get(url)
        first_soup = BeautifulSoup(response.text, 'html.parser')# get the html soupt
        heading = first_soup.find_all('h1', class_ = "it-ttl")#find the item description
        seller = first_soup.find_all('span', class_ = "mbg-nw")#find the seller name
        seller_feedback = first_soup.find_all('span', class_ = "mbg-l")#get the seller feedback
        no_items_sold = first_soup.find_all('span', class_ = "soldwithfeedback")#get the total no. of items sold
        rating = first_soup.find_all('a', class_ = "reviews-star-rating")#find item rating if any
        item_description.append(heading[0].text[16:])
        seller_name.append(seller[0].text)
        sellerfeedback.append(np.int(seller_feedback[0].text[15:-2]))
        item = first_soup.find_all('span', itemprop = "name")
        available_quantity = first_soup.find_all('span', id = "qtySubTxt")
        retail_price = first_soup.find_all('span', id = "prcIsum")#find the retail price
        discount_price = first_soup.find_all('span', id = "mm-saleDscPrc")
        try:#append retail price, look for error
            y=np.float(retail_price[0].attrs['content'])
            price.append(y)
            print('The price is {}'.format(y))
        except:#else look for discounted price if any
            z  = discount_price[0].text
            m = np.float(''.join(i for i in z if (i.isdigit() or i=='.')))
            price.append(m) 
            print('The discounted price is {}'.format(m))
        try:#append currency 
            price_string = retail_price[0].text.replace('.','')
            currency_1 = ''.join([i for i in price_string if not i.isdigit()])
            currency.append(currency_1)
            print('The currency is {}'.format(currency_1))
        except:#append currency for discounted price
            price_string = discount_price[0].text.replace('.','')
            currency_1 = ''.join([i for i in price_string if not i.isdigit()])
            currency.append(currency_1)
            print('The currency is {}'.format(currency_1))
        try:
            x = available_quantity[0].text
            quantity = [int(s) for s in x.split() if s.isdigit()][0]
            quantity_available.append(quantity)
            print('Available quantity is : {}'.format(quantity))
        except:
            quantity_available.append(np.nan)
        try:
            keyword.append(item[4].text)
            print(item[4].text)
        except:
            if len(item)>0:
                keyword.append(item[-2].text)
                print(item[-2].text)
            else:
                keyword.append('Not available')
                print('Key word not available')
        try:
            no_of_sold_items.append(np.int(no_items_sold[0].text[6:-25].replace(",","")))
            
        except: #IndexError:
            no_of_sold_items.append(np.nan)
        try:# because in some cases table numbers are different, we have table numbers 5, 6 or 10
            last_sale_url = no_items_sold[0].a.attrs['href']
            response_sale = get(last_sale_url)
            html_salesoup = BeautifulSoup(response_sale.text, 'html.parser')
            table = html_salesoup.find_all('table')# extract all tables from the soup
            try:
                sale_rate =sale_rate_tables(table, table_no=5)
                rate_of_sale.append(sale_rate)
            except:
                try:
                    
                    sale_rate = sale_rate_tables(table, table_no=6)
                    rate_of_sale.append(sale_rate)
                except:
                    sale_rate = sale_rate_tables(table, table_no=10)
                    rate_of_sale.append(sale_rate)
        except:
            rate_of_sale.append(np.nan)
        try:
            item_rating.append(np.float(rating[0].attrs['aria-label'][:3]))
            
        except IndexError:
            item_rating.append(np.nan)
            
    sale_item_desc = {'Keyword':keyword, 'Description':item_description, 'seller name':seller_name, 'feedback':sellerfeedback,
                      'Total no. sold':no_of_sold_items, 'rating':item_rating, 'Rate of Sale(per day)':rate_of_sale, 
                     'Quantity Available':quantity_available, 'Price':price, 'Currency':currency}
    return pd.DataFrame(sale_item_desc)

In [29]:
data = get_item_details(all_items_link)

https://www.ebay.com/itm/50000lm-Military-T6-LED-Flashlight-Torch-5-Modes-Zoom-Tactical-18650-Lamp-Light/372283200530?epid=2222957697&hash=item56adced412:g:SkkAAOSwU4FZ6ZCs
The discounted price is 3.98
The currency is US $
Available quantity is : 10
Flashlights


/Users/kiran/anaconda3/lib/python3.6/site-packages/dateutil/parser/_parser.py:1204: UnknownTimezoneWarning: tzname PDT identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  category=UnknownTimezoneWarning)


182 47
https://www.ebay.com/itm/Mini-5000lm-Q5-LED-Tactical-Shadowhawk-X800-Flashlight-Aluminium-Torch-new/312118553320?epid=16011128007&hash=item48abb72ae8:g:MFgAAOSwevlaFa8N
The price is 1.29
The currency is C $
Available quantity is : 10
Flashlights
https://www.ebay.com/itm/Inflatable-Air-Sofa-Bed-Lazy-Sleeping-Camping-Bag-Beach-Hangout-Couch-Windbed-18/222969236800?epid=715030241&hash=item33ea008d40:m:mSeLLE_HXu8OCpTWY6uyAZA&var=521774423593
The discounted price is 2.29
The currency is US $
Available quantity is : 10
Sleeping Bags
111 6
https://www.ebay.com/itm/25-50-100FT-550-Paracord-Rope-7-strand-Parachute-CordCAMPING-HiKING/173323995376?hash=item285aea48f0:m:mlfroseGfePee76ORN6V5GQ&var=472035693953
The price is 1.29
The currency is US $
Available quantity is : 10
Paracord Bracelets
104 5
https://www.ebay.com/itm/Outdoor-Camping-Hiking-Survival-Bag-Travel-Emergency-Rescue-First-Aid-Kit-New/202181722317?epid=594562826&hash=item2f12f840cd:g:1cEAAOSwE8daValt
The price is 0.99
The c

The price is 3.19
The currency is US $
Available quantity is : 10
Other Camping/Hiking Backpacks
183 104
https://www.ebay.com/itm/New-Color-50-100FT-2mm-Paracord-Micro-Cord-Parachute-Cord-Tent-Lanyard-Rope/173279158779?hash=item28583e21fb:m:mBABbgGD1ZNwdJX4KlBqzww&var=471988850077
The price is 1.97
The currency is US $
Available quantity is : 10
Paracord Bracelets
108 13
https://www.ebay.com/itm/Outdoor-Portable-Camping-Climbing-Hunting-Knife-EDC-Survival-Self-defence-Tool-E/232805878958?hash=item36344fccae:m:m4ci74H1D85F2R1tHFbvbqQ&var=532317599066
The price is 2.41
The currency is US $
Available quantity is : 10
Pocket, Multi Tools
https://www.ebay.com/itm/Outdoor-Camping-Hiking-Survival-Fire-Starter-Flint-Match-Metal-Lighter-Emergenc/332624339608?hash=item4d71f4a298:g:VloAAOSwll1WwaJN
The discounted price is 1.25
The currency is C $
Available quantity is : 10
Other Emergency Gear
240 42
https://www.ebay.com/itm/New-Multi-Tool-Wrench-Jaw-Screwdrive-Pliers-Knife-Set-Survival-Gear-Adju

In [30]:
data

,Currency,Description,Keyword,Price,Quantity Available,Rate of Sale(per day),Total no. sold,feedback,rating,seller name
0,US $,50000lm Military T6 LED Flashlight Torch 5 Mod...,Flashlights,3.98,10,3.872340,258.0,51182,4.2,topsun66
1,C $,Mini 5000lm Q5 LED Tactical Shadowhawk X800 F...,Flashlights,1.29,10,NaN,NaN,19444,4.3,flytosky201678
2,US $,Inflatable Air Sofa Bed Lazy Sleeping Camping ...,Sleeping Bags,2.29,10,18.500000,NaN,1060,NaN,scc666
3,US $,25/50/100FT 550 Paracord Rope 7 strand Parach...,Paracord Bracelets,1.29,10,20.800000,389.0,1834,NaN,yangyu19870915-8
4,US $,Outdoor Camping Hiking Survival Bag Travel Eme...,Other Emergency Gear,0.99,10,NaN,NaN,50490,4.2,fashion-space7
5,C $,Camping Emergency Rod Flint Stone Fire Starter...,Fire Starters,1.06,10,35.000000,4121.0,36400,NaN,zjchen19850703
6,GBP,Food Ration MILITARY US ARMY MRE Pack Emergenc...,MREs & Freeze-Dried Food,13.95,10,2.140625,1342.0,410,4.8,mrefood
7,US $,Portable WindProof Storm Matches Emergency Sur...,Other Emergency Gear,0.99,10,10.000000,NaN,85450,5.0,buyeasy-uk2
8,US $,Outdoor Emergency Tent Blanket Sleeping Bag Su...,Tents,0.99,10,8.178571,3002.0,8298,4.6,carmarket2014
9,US $,10Yards Multicolor Parachute Cord Lanyard Rope...,Paracord Bracelets,0.99,10,3.642857,106.0,51606,NaN,zero*one01


In [31]:
data.to_csv('sample_data.csv', index = False)